# Demographic data from the American Community Survey

**[Work in progress]**

This notebook downloads [selected demographic data (DP05)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP05) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


## Download selected variables

* [Selected demographic estimates for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP05)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP05.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP05/)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP05 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

The numbers in each subgroup below adds up to the total population

In [4]:
variables = {'DP05_0001E': 'totalPopulation',
             
             # sex
             'DP05_0002E': 'male',
             'DP05_0003E': 'female',
             
             # age
             'DP05_0005E': 'age0_4',
             'DP05_0006E': 'age5_9',
             'DP05_0007E': 'age10_14',
             'DP05_0008E': 'age15_19',
             'DP05_0009E': 'age20_24',
             'DP05_0010E': 'age25_34',
             'DP05_0011E': 'age35_44',
             'DP05_0012E': 'age45_54',
             'DP05_0013E': 'age55_59',
             'DP05_0014E': 'age60_64',
             'DP05_0015E': 'age65_74',
             'DP05_0016E': 'age75_84',
             'DP05_0017E': 'age85_',

              # race
             'DP05_0037E': 'white',
             'DP05_0038E': 'blackOrAfricanAmerican',
             'DP05_0039E': 'americanIndianAndAlaskaNative',
             'DP05_0044E': 'asian',
             'DP05_0052E': 'nativeHawaiianAndOtherPacificIslander',
             'DP05_0057E': 'otherRace',
             'DP05_0058E': 'twoOrMoreRaces',
             
              # ethnicity
             'DP05_0071E': 'hispanicOrLatino',
             'DP05_0076E': 'notHispanicOrLatino'
            }

In [5]:
fields = ",".join(variables.keys())

## Download county-level data using US Census API

In [6]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [7]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,DP05_0001E,DP05_0002E,DP05_0003E,DP05_0005E,DP05_0006E,DP05_0007E,DP05_0008E,DP05_0009E,DP05_0010E,DP05_0011E,DP05_0012E,DP05_0013E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0052E,DP05_0057E,DP05_0058E,DP05_0071E,DP05_0076E,state,county
1,47086,22018,25068,3614,3107,3554,3152,3351,5844,5236,5760,3224,3291,4282,1961,710,12097,33982,56,282,0,391,278,729,46357,28,151
2,12028,5845,6183,728,776,806,849,671,1279,1562,1602,786,897,1247,589,236,9475,2368,0,17,0,16,152,181,11847,28,111
3,8321,4174,4147,458,564,486,598,416,866,921,1092,536,644,979,547,214,5631,2596,23,15,8,13,35,33,8288,28,019
4,23480,11821,11659,1262,1439,1506,1919,1670,2753,2830,3142,1519,1435,2250,1181,574,21346,1681,75,57,7,170,144,360,23120,28,057


##### Add column names

In [8]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [9]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [10]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [11]:
df.head()

,totalPopulation,male,female,age0_4,age5_9,age10_14,age15_19,age20_24,age25_34,age35_44,age45_54,age55_59,age60_64,age65_74,age75_84,age85_,white,blackOrAfricanAmerican,americanIndianAndAlaskaNative,asian,nativeHawaiianAndOtherPacificIslander,otherRace,twoOrMoreRaces,hispanicOrLatino,notHispanicOrLatino,stateFips,countyFips
1,47086,22018,25068,3614,3107,3554,3152,3351,5844,5236,5760,3224,3291,4282,1961,710,12097,33982,56,282,0,391,278,729,46357,28,151
2,12028,5845,6183,728,776,806,849,671,1279,1562,1602,786,897,1247,589,236,9475,2368,0,17,0,16,152,181,11847,28,111
3,8321,4174,4147,458,564,486,598,416,866,921,1092,536,644,979,547,214,5631,2596,23,15,8,13,35,33,8288,28,019
4,23480,11821,11659,1262,1439,1506,1919,1670,2753,2830,3142,1519,1435,2250,1181,574,21346,1681,75,57,7,170,144,360,23120,28,057
5,10129,5116,5013,455,467,740,803,416,812,1212,1390,962,623,1353,726,170,6523,3503,6,0,0,0,97,29,10100,28,015


In [12]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,totalPopulation,male,female,age0_4,age5_9,age10_14,age15_19,age20_24,age25_34,age35_44,age45_54,age55_59,age60_64,age65_74,age75_84,age85_,white,blackOrAfricanAmerican,americanIndianAndAlaskaNative,asian,nativeHawaiianAndOtherPacificIslander,otherRace,twoOrMoreRaces,hispanicOrLatino,notHispanicOrLatino,stateFips,countyFips
1869,3302833,1661931,1640902,211969,198148,197726,209496,262118,541385,436855,420221,201666,183654,251516,127904,60175,2335447,166412,20980,390418,13903,205307,170366,1106925,2195908,06,073


In [13]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [14]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP05Admin2.csv", index=False)

##### Data Checks for US
50 states + District of Columbia

In [15]:
print('Number of states:', len(stateFips)) 

Number of states: 51


In [16]:
df['totalPopulation'] = df['totalPopulation'].astype(int)
df['totalPopulation'].sum()

322903030

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,DP05_0001E,DP05_0002E,DP05_0003E,DP05_0005E,DP05_0006E,DP05_0007E,DP05_0008E,DP05_0009E,DP05_0010E,DP05_0011E,DP05_0012E,DP05_0013E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0052E,DP05_0057E,DP05_0058E,DP05_0071E,DP05_0076E,zip code tabulation area
1,8642,4312,4330,567,430,453,532,344,901,975,1226,843,745,916,522,188,8226,256,2,21,0,0,137,27,8615,43964
2,51116,23286,27830,3438,3367,3204,4094,3740,7553,8183,7607,2628,2225,2958,1577,542,13583,32929,25,1054,0,2274,1251,4077,47039,28216
3,71605,33421,38184,4222,5769,5767,4563,2944,8540,11786,11186,5117,4079,5063,1842,727,51272,6346,200,10533,0,978,2276,5172,66433,28277
4,27286,12870,14416,2277,1988,1861,1812,1339,4140,4488,3950,1806,1336,1593,533,163,14515,8672,7,1709,53,590,1740,2870,24416,28278


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,totalPopulation,male,female,age0_4,age5_9,age10_14,age15_19,age20_24,age25_34,age35_44,age45_54,age55_59,age60_64,age65_74,age75_84,age85_,white,blackOrAfricanAmerican,americanIndianAndAlaskaNative,asian,nativeHawaiianAndOtherPacificIslander,otherRace,twoOrMoreRaces,hispanicOrLatino,notHispanicOrLatino,postalCode
1,8642,4312,4330,567,430,453,532,344,901,975,1226,843,745,916,522,188,8226,256,2,21,0,0,137,27,8615,43964
2,51116,23286,27830,3438,3367,3204,4094,3740,7553,8183,7607,2628,2225,2958,1577,542,13583,32929,25,1054,0,2274,1251,4077,47039,28216
3,71605,33421,38184,4222,5769,5767,4563,2944,8540,11786,11186,5117,4079,5063,1842,727,51272,6346,200,10533,0,978,2276,5172,66433,28277
4,27286,12870,14416,2277,1988,1861,1812,1339,4140,4488,3950,1806,1336,1593,533,163,14515,8672,7,1709,53,590,1740,2870,24416,28278
5,29414,13958,15456,2144,1784,1542,1504,2318,5357,3591,2955,1842,1673,2610,1406,688,14358,11486,187,1185,132,345,1721,3084,26330,28303


In [21]:
# Example data
df.query("postalCode == '90210'")

,totalPopulation,male,female,age0_4,age5_9,age10_14,age15_19,age20_24,age25_34,age35_44,age45_54,age55_59,age60_64,age65_74,age75_84,age85_,white,blackOrAfricanAmerican,americanIndianAndAlaskaNative,asian,nativeHawaiianAndOtherPacificIslander,otherRace,twoOrMoreRaces,hispanicOrLatino,notHispanicOrLatino,postalCode
30897,19909,9599,10310,710,1014,1482,1066,519,1956,2292,3143,1282,1093,2562,1695,1095,16911,198,21,1782,0,313,684,829,19080,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP05Zip.csv", index=False)

##### Check Data

Note, this includes zip codes for Puerto Rico

In [24]:
df['totalPopulation'] = df['totalPopulation'].astype(int)
df['totalPopulation'].sum()

326274356

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [25]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.spleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [26]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [27]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [28]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [29]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,totalPopulation,male,female,age0_4,age5_9,age10_14,age15_19,age20_24,age25_34,age35_44,age45_54,age55_59,age60_64,age65_74,age75_84,age85_,white,blackOrAfricanAmerican,americanIndianAndAlaskaNative,asian,nativeHawaiianAndOtherPacificIslander,otherRace,twoOrMoreRaces,hispanicOrLatino,notHispanicOrLatino,stateFips,countyFips,tract,source,aggregationLevel
56,7222,3701,3521,300,744,346,135,199,710,1093,1137,530,567,1049,320,92,6038,16,0,794,12,77,285,532,6690,06,073,06073008324,American Community Survey 5 year,Tract
57,2053,1195,858,79,42,47,107,778,479,262,151,39,23,31,10,5,1154,56,0,614,3,134,92,324,1729,06,073,06073008339,American Community Survey 5 year,Tract
58,7037,3708,3329,461,510,360,317,206,1237,970,943,488,449,752,316,28,2388,216,12,3166,29,551,675,974,6063,06,073,06073008347,American Community Survey 5 year,Tract
59,9862,4507,5355,958,561,380,347,674,2658,1434,1336,366,411,423,200,114,2812,452,0,4791,121,535,1151,1489,8373,06,073,06073008354,American Community Survey 5 year,Tract
60,6576,3398,3178,387,329,192,194,675,1130,1005,955,518,447,326,180,238,3914,178,26,1655,0,562,241,1791,4785,06,073,06073008505,American Community Survey 5 year,Tract


### Save data

In [30]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP05Tract.csv", index=False)

In [31]:
df.shape

(73056, 30)